# Adapters

> Adapter implementations for integrating with plugin registries

In [ ]:
#| default_exp core.adapters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, List, Optional

from cjm_fasthtml_workflow_transcription_single_file.core.registry import UnifiedPluginRegistry
from cjm_fasthtml_workflow_transcription_single_file.core.protocols import PluginInfo, PluginRegistryProtocol
from cjm_plugin_system.utils.validation import extract_defaults

## PluginRegistryAdapter Class

Adapter that wraps a `UnifiedPluginRegistry` (from `cjm-fasthtml-plugins`) to implement `PluginRegistryProtocol`. This allows the workflow to work with any plugin registry that follows the unified plugin registry interface.

In [ ]:
#| export
class PluginRegistryAdapter:
    """Adapts app's UnifiedPluginRegistry to workflow's PluginRegistryProtocol."""

    def __init__(self,
                 app_registry,  # The app's UnifiedPluginRegistry instance
                 category: str = "transcription"  # Plugin category to filter by
                 ):
        """Initialize the adapter."""
        self._registry = app_registry
        self._category = category

    def get_configured_plugins(self) -> List[PluginInfo]:  # List of PluginInfo for configured plugins
        """Get all configured transcription plugins (those with saved config files)."""
        plugins = self._registry.get_plugins_by_category(self._category)
        return [
            PluginInfo(
                id=p.get_unique_id(),
                name=p.name,
                title=p.title,
                is_configured=p.is_configured,
                supports_streaming=self._check_streaming_support(p)
            )
            for p in plugins if p.is_configured
        ]

    def get_all_plugins(self) -> List[PluginInfo]:  # List of PluginInfo for all discovered plugins
        """Get all discovered transcription plugins (configured or not)."""
        plugins = self._registry.get_plugins_by_category(self._category)
        return [
            PluginInfo(
                id=p.get_unique_id(),
                name=p.name,
                title=p.title,
                is_configured=p.is_configured,
                supports_streaming=self._check_streaming_support(p)
            )
            for p in plugins
        ]

    def get_plugin(self,
                   plugin_id: str  # Unique plugin identifier
                   ) -> Optional[PluginInfo]:  # PluginInfo if found, None otherwise
        """Get a specific plugin by ID."""
        plugin_meta = self._registry.get_plugin(plugin_id)
        if not plugin_meta:
            return None

        return PluginInfo(
            id=plugin_meta.get_unique_id(),
            name=plugin_meta.name,
            title=plugin_meta.title,
            is_configured=plugin_meta.is_configured,
            supports_streaming=self._check_streaming_support(plugin_meta)
        )

    def get_plugin_config(self,
                          plugin_id: str  # Unique plugin identifier
                          ) -> Dict[str, Any]:  # Configuration dictionary, empty dict if not configured
        """Get the configuration for a plugin."""
        return self._registry.load_plugin_config(plugin_id) or {}

    def _check_streaming_support(self,
                                 plugin_meta  # Plugin metadata object
                                 ) -> bool:  # True if the plugin supports streaming
        """Check if a plugin supports streaming output."""
        if hasattr(plugin_meta, 'config_schema'):
            schema = plugin_meta.config_schema
            if isinstance(schema, dict):
                properties = schema.get("properties", {})
                return "stream_output" in properties or "streaming" in properties
        return False

## DefaultConfigPluginRegistryAdapter Class

Extended adapter that applies default configuration values when loading plugin configs. This ensures all required fields have values even if not explicitly configured.

In [ ]:
#| export
class DefaultConfigPluginRegistryAdapter(PluginRegistryAdapter):
    """Plugin registry adapter that provides default config values for unconfigured plugins."""

    def __init__(self,
                 registry: UnifiedPluginRegistry,  # The UnifiedPluginRegistry instance to wrap
                 category: str = "transcription"  # Plugin category to filter by
                 ):
        """Initialize adapter with registry instance."""
        self._registry = registry
        self._category = category

    def get_plugins_by_category(self,
                                category: str  # Plugin category to filter by
                                ) -> list:  # List of plugins in the category
        """Get all plugins in a specific category."""
        return self._registry.get_plugins_by_category(category)

    def get_plugin(self,
                   plugin_id: str  # Unique plugin identifier
                   ):  # Plugin metadata or None
        """Get a specific plugin by ID."""
        return self._registry.get_plugin(plugin_id)

    def load_plugin_config(self,
                           plugin_id: str  # Unique plugin identifier
                           ) -> Dict[str, Any]:  # Configuration dictionary with defaults applied
        """Load configuration for a plugin, using defaults if no saved config exists."""
        # First try to load saved config
        saved_config = self._registry.load_plugin_config(plugin_id)

        # If config exists and has values, return it
        if saved_config:
            return saved_config

        # No saved config - get default values from config dataclass
        plugin_meta = self._registry.get_plugin(plugin_id)
        if plugin_meta and hasattr(plugin_meta, 'config_class') and plugin_meta.config_class is not None:
            return extract_defaults(plugin_meta.config_class)

        # Fallback to empty dict
        return {}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()